<a href="https://colab.research.google.com/github/BeautifulTovarisch/statistics-potpourri/blob/main/basic_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Plots

This Notebook demonstrates basic plotting with Julia using the PyPlot backend.

In [ ]:
"""
Packages

Note: Run this with caution, as it may take 20+ minutes
"""

import Pkg

Pkg.add("Plots")
Pkg.add("Images")
Pkg.add("PyPlot")
Pkg.add("Measures")
Pkg.add("LaTeXStrings")

## Introduction

The following cells plot a simple polynomial and render a heatmap.

In [ ]:
using Plots, LaTeXStrings, Measures; pyplot()

f(x,y) = x^2 + y^2
f0(x) = f(x, 0)
f2(x) = f(x, 2)

xVals, yVals = -5:0.1:5, -5:0.1:5
plot(xVals, [f0.(xVals), f2.(xVals)],
      c=[:blue :red], xlims=(-5,5), legend=:top,
      ylims=(-5,25), ylabel=L"f(x,\cdot)", label=[L"f(x,0)" L"f(x,2)"])

# p1 = annotate!(0, -0.2, text("(0,0) The minimum\n of f(x,0)", :left, :top, 10))
#
# z = [f(x,y) for y in yVals, x in xVals]
# p2 = surface(xVals, yVals, z, c=cgrad([:blue, :red]),legend=:none,
#       ylabel="y", zlabel=L"f(x,y)")
#
# M = z[1:10,1:10]
# p3 = heatmap(M, c=cgrad([:blue, :red]), yflip=true, ylabel="y",
# xticks=([1:10;], xVals), yticks=([1:10;], yVals))
#
# plot(p1, p2, p3, layout=(1,3), size=(1200,400), xlabel="x", margin=5mm)

## Hailstone Sequence

Suppose we want to generate a sequence of integers such that $x_{n+1} = f(x_n)$, where:

\begin{align}
f(x) =
\left\{
  \begin{array}{cl}
    x/2 & \text{if } x \mod 2 = 0 \\
    3x + 1 & \text{otherwise}
  \end{array}
  \right.
\end{align}

This sequence is known as the _hailstone_ sequence. The _Collatz conjecture_ posits that no matter how $x_0$ is chosen, the hailstone sequence converges at 1.

In [ ]:
"""
Hailstone Sequence Histogram

The following plots the hailstone sequence given by the formula above.
"""

using Plots; pyplot()

"""
hailLength(x0) computes the number of elements in a hailstone sequence starting
at `x0`.

`x0` must be a positive Int.
"""
function hailLength(x0::Int)
  len = 0
  while x0 > 1
    x0 = (x0 % 2 == 1) ? 3x0 + 1 : Int(x0/2)

    len += 1
  end

  return len
end

lens = [hailLength(x0) for x0 in 2:10^7]

histogram(lens, bins=1000, normed=:true,
          fill=(:blue, true), la=0, legend=:none,
          xlims=(0, 500), ylims=(0, 0.012),
          xlabel="Length", ylabel="Frequency")

## Animations

The following cells demonstrate a simple animation of a mathematical graph. The animation draws equally spaced vertices around the unit circle and obtain a sequence of complex numbers given by
$z_n = e^{2\pi i \frac{k}{n}}$


In [ ]:
"""
Animation

This cell renders equally spaced vertices around the unit circle and draws edges
between these vertices.
"""

using Plots; pyplot()

"""
graphCreator(n) constructs `n` equally spaced vertices about the unit circle and
animates drawing edges between these vertices to form a complete graph.
"""
function graphCreator(n::Int)
  vertices = 1:n

  # Use the real and complex parts of these numbers as the coordinates
  # TODO: Combine comprehensions to avoid wasted allocation
  complexPts = [exp(2*pi*im * (k/n)) for k in vertices]
  coords = [(real(p), imag(p)) for p in complexPts]

  xPts = first.(coords)
  yPts = last.(coords)

  edges = [(v, u) for v in vertices for u in (v+1):n]

  # Construct the animation and plot the vertices
  anim = Animation()
  scatter(xPts, yPts, c=:blue, msw=0, ratio=1,
    xlims=(-1.5,1.5), ylims=(-1.5, 1.5), legend=:none)

  # Animate drawing the edges
  for edge in edges
    u, v = edge[1], edge[2]

    xpoints = [xPts[u], xPts[v]]
    ypoints = [yPts[u], yPts[v]]

    plot!(xpoints, ypoints, line=(:red))

    frame(anim)
  end

  gif(anim, "graph.gif", fps=60)
end

graphCreator(10)

## Rastering Stars

This example passes a smoothing function over an image of space in order to reduce noise and identify the _true_ brightest star.

In [ ]:
using Plots, Images; pyplot()

img = load("./stars.png")
gImg = red.(img)*0.299 + blue.(img)*0.587 + green.(img)*0.177

rows, cols = size(img)

# println("Highest intensity pixel: ", findmax(gImg))

function boxBlur(image,x,y,d)
  if x<=d || y<=d || x>=cols-d || y>=rows-d
    return image[x,y]
  else
    total = 0.0
    for xi = x-d:x+d
      for yi = y-d:y+d
        total += image[xi, yi]
      end
    end

    return total/((2d+1)^2)
  end
end

blurImg = [boxBlur(gImg, x, y, 5) for x in 1:cols, y in 1:rows]
yOriginal, xOriginal = argmax(gImg).I
yBoxBlur, xBoxBlur = argmax(blurImg).I

p1 = heatmap(gImg, c=:Greys, yflip=true)
p1 = scatter!((xOriginal, yOriginal), ms=60, ma=0, msw=4, msc=:red)
p2 = heatmap(blurImg, c=:Greys, yflip=true)
p2 = scatter!((xBoxBlur, yBoxBlur), ms=60, ma=0, msw=4, msc=:red)

plot(p1, p2, size=(800, 400), ratio=:equal, xlims=(0,cols), ylims=(0,rows),
colorbar_entry=false, border=:none, legend=:none)